# modulos

In [1]:
import numpy as np
import pandas as pd
import re

import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'notebook'

import matplotlib.pyplot as plt
import seaborn as sns

import requests
from  bs4 import BeautifulSoup
import urllib

from selenium import webdriver
from selenium.webdriver.common.by import By
#Buscar y colocar ChromeDriver en la ruta del archivo a trabajar
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

from wordcloud import WordCloud, STOPWORDS
from PIL import Image

from time import sleep
from pprint import pprint

import folium
import geopandas as gpd
from tqdm import tqdm
import json

# Web Scrapping

## supermercado mas

In [125]:
url_mas = "https://www.supermercadosmas.com/"

chrome_driver = "chromedriver"

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
browser = webdriver.Chrome(options=options)
browser.get(url = url_mas)

wait = WebDriverWait(browser, 40)
wait.until(EC.element_to_be_clickable((By.ID, "btn-cookie-allow"))).click()
sleep(4)
buscador = browser.find_element(By.NAME, "check_rdgl_cp")
buscador.send_keys("41001")
sleep(1)
boton_consultar = wait.until(EC.presence_of_element_located((By.XPATH, "//span[text()='Consultar']")))
browser.execute_script("arguments[0].click();", boton_consultar)
sleep(10)
continuar_boton = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[title='Continuar']")))
continuar_boton.click()

try:
    ventana_emergente = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a[href='#__cn_close_content']")))
    ventana_emergente.click()
except EC.TimeoutException:
    print("La ventana emergente no apareció.")

frutas_elemento = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[text()='Frutas']"))).click()
sleep(7)

data_mas = []

for pagina in range(2):
    # Realizar scroll hacia el botón "Ver siguientes"
    browser.execute_script("arguments[0].scrollIntoView();", wait.until(EC.visibility_of_element_located((By.XPATH, "//h3[text()='Ver siguientes']"))))

    # Hacer clic en el botón "Ver siguientes" utilizando execute_script()
    ver_siguientes = wait.until(EC.element_to_be_clickable((By.XPATH, "//h3[text()='Ver siguientes']")))
    browser.execute_script("arguments[0].click();", ver_siguientes)

    sleep(2)

soup = BeautifulSoup(browser.page_source, "html.parser")

productos = soup.find_all("a", class_="product-item-link")
precios = soup.find_all("div", class_="product-item-relative-units")

nombres_productos = [producto.text for producto in productos]
precios_productos = [float(precio.text.split()[0]) for precio in precios]

for nombre, precio in zip(nombres_productos, precios_productos):
    data_mas.append({"nombre_supermercadosmas": nombre, "precio_supermercadosmas":precio, 'supermercadosmas':'supermercadosmas'})

browser.quit()

Airtable Supermercadosmas

In [126]:
API_KEY = "" # Usuario

BASE_ID = "app3rDVq7YZmHLPMn" # Base: Tabla API

TABLE_ID = "tbl6FCpAlIrAgkpWO" # Tabla: datos_1

airtable_base_url = "https://api.airtable.com/v0"

# Headers
headers = {"Authorization" : f"Bearer {API_KEY}",
           "Content-Type"  : "application/json"}

endpoint = f"{airtable_base_url}/{BASE_ID}/{TABLE_ID}"

records = []
for item in data_mas:
    records_item = {'fields': {"Producto": item["nombre_supermercadosmas"], 'Precio': item['precio_supermercadosmas'],'Supermercado':item['supermercadosmas']}}
    records.append(records_item)
    
cantidad_records = len(records)
posicion = 0

while posicion < cantidad_records:
    records_utilizar = records[posicion : (posicion + 10)]
    records_dict = {"records" : records_utilizar}
    response = requests.post(url = endpoint, json = records_dict, headers = headers)

    if response.status_code != 200:
        print(response.text)
        break
    
    posicion += 10

## Supermercado Carrefour

In [117]:
url_carrefour = "https://www.carrefour.es/"
chrome_driver = "chromedriver"

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
browser = webdriver.Chrome(executable_path=chrome_driver, options=options)
browser.get(url=url_carrefour)

wait = WebDriverWait(browser, 10)

# Acceptar cookies y navegar
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#onetrust-accept-btn-handler"))).click()
button = browser.find_element(By.XPATH, '//button[@aria-label=" Modificar"]').click()
input_element = browser.find_element(By.CLASS_NAME, 'sale-point__postal-code-input')
input_element.send_keys('41001')
browser.find_element(By.CSS_SELECTOR, '.options-buttons__accept').click()
browser.find_element(By.CSS_SELECTOR, '.options-buttons__accept').click()
wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@title="Supermercado"]'))).click()
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "icon-cross-thin"))).click()
wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@title="Productos Frescos"]'))).click()
wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@title="Frutas"]'))).click()


data_carrefour = []

# Define el número de páginas a recorrer
num_paginas = 5

for pagina in range(num_paginas):
    # Scroll para ver todos los productos
    browser.execute_script("window.scrollBy(0, window.innerHeight);")
    sleep(0.5)

    # Cogemos los productos y extraemos nombre y precio
    soup = BeautifulSoup(browser.page_source, "html.parser")
    #productos = soup.find_all(lambda tag: tag.name == "h2" and re.search(r'\bproduct\b', str(tag.get('class', []))))
    productos = soup.find_all('div', class_ = "product-card")
    

    for producto in productos:
        nombre = producto.find("a", class_="product-card__title-link track-click").text.split()[0]
        precio = producto.find("span", class_="product-card__price-per-unit")
        
        if precio:
            precio = re.sub(r'[^\d,]', '', precio.text.strip())
            precio = float(precio.replace(",", "."))
            
            
        else:
            precio = "None"
            
        data_carrefour.append({"nombre_carrefour": nombre, "precio_carrefour": precio, 'carrefour':'carrefour'})

    if pagina < num_paginas - 1:
        # Clickamos para ir a la siguiente página
        next_page_button = browser.find_element(By.CLASS_NAME, "pagination__next")
        next_page_button.click()
        sleep(2)  

# Cerrar el navegador después de completar el scraping
browser.quit()

In [47]:
def eliminar_no_disponible(lista):
    for i in range(len(lista) - 1, -1, -1):
        if lista[i]['precio_carrefour'] == 'None':
            del lista[i]

In [48]:
eliminar_no_disponible(data_carrefour)

Airtable Carrefour

In [122]:
API_KEY = "" # Usuario

BASE_ID = "app3rDVq7YZmHLPMn" # Base: Tabla API

TABLE_ID = "tbl6FCpAlIrAgkpWO" # Tabla: datos_1

airtable_base_url = "https://api.airtable.com/v0"

# Headers
headers = {"Authorization" : f"Bearer {API_KEY}",
           "Content-Type"  : "application/json"}

endpoint = f"{airtable_base_url}/{BASE_ID}/{TABLE_ID}"

records = []
for item in data_carrefour:
    records_item = {'fields': {"Producto": item["nombre_carrefour"], 'Precio': item['precio_carrefour'],'Supermercado':item['carrefour']}}
    records.append(records_item)
    
cantidad_records = len(records)
posicion = 0

while posicion < cantidad_records:
    records_utilizar = records[posicion : (posicion + 10)]
    records_dict = {"records" : records_utilizar}
    response = requests.post(url = endpoint, json = records_dict, headers = headers)

    if response.status_code != 200:
        print(response.text)
        break
    
    posicion += 10

## Supermercado Mercadona

In [ ]:
browser = webdriver.Chrome()

browser.get("https://tienda.mercadona.es/categories/27")

browser.maximize_window()

#Entrar a una tienda por codigo postal
#Crear elemento soup para trabajar con el
soup = BeautifulSoup(browser.page_source, "html.parser")
buscador = browser.find_element(By.CSS_SELECTOR,"#root > div.cookie-banner > div > div > button.ui-button.ui-button--small.ui-button--primary.ui-button--positive").click()
buscador = browser.find_element(By.CLASS_NAME, "ym-hide-content")
buscador.send_keys('20100')
sleep(1)
buscador = browser.find_element(By.CSS_SELECTOR,"#root > div.ui-focus-trap > div > div:nth-child(2) > div > form > button").click()

#Crear elemento soup para trabajar con él
soup = BeautifulSoup(browser.page_source, "html.parser")

productos = browser.find_elements(By.CLASS_NAME, "product-cell")

#Iterar en productos mercadona para extraer url y el id del producto
lista_id = []
for producto in productos:
    
    sleep(1)
    producto.click()
    sleep(1)
    url = browser.current_url
    lista_id.append(url.split('/')[-2])
    browser.back()
browser.quit()
    
#Recopilación de datos mediante el id
data_mercadona = []
for idx in lista_id:
    browser = webdriver.Chrome()

    browser.get(f"https://tienda.mercadona.es/api/products/{idx}/")
    browser.maximize_window()
    soup = BeautifulSoup(browser.page_source, "html.parser")
    
    try:
        json_text = soup.find("pre").get_text()
        data = json.loads(json_text)
    except:
        pass
    try:
        precio = float(data["price_instructions"]["bulk_price"])
        nombre = data["display_name"].split()[0]
        
        data_mercadona.append({"nombre": nombre, "precio": precio, "mercadona":"mercadona"})
    except:
        pass
    browser.quit()
    

Airtable Mercadona

In [ ]:
API_KEY = "" # Usuario

BASE_ID = "app3rDVq7YZmHLPMn" # Base: Tabla API

TABLE_ID = "tbl6FCpAlIrAgkpWO" # Tabla: datos_1

airtable_base_url = "https://api.airtable.com/v0"

# Headers
headers = {"Authorization" : f"Bearer {API_KEY}",
           "Content-Type"  : "application/json"}

endpoint = f"{airtable_base_url}/{BASE_ID}/{TABLE_ID}"

records = []
for item in data_mercadona:
    records_item = {'fields': {"Producto": item["nombre"], 'Precio': item['precio'],'Supermercado':item['mercadona']}}
    records.append(records_item)
    
cantidad_records = len(records)
posicion = 0

while posicion < cantidad_records:
    records_utilizar = records[posicion : (posicion + 10)]
    records_dict = {"records" : records_utilizar}
    response = requests.post(url = endpoint, json = records_dict, headers = headers)

    if response.status_code != 200:
        print(response.text)
        break
    
    posicion += 10

## Supermercado Eroski

In [ ]:
from time import sleep, time

url_eroski = "https://supermercado.eroski.es/"

chrome_driver = "chromedriver"

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
browser = webdriver.Chrome(executable_path = chrome_driver, options=options)
browser.get(url = url_eroski)

wait = WebDriverWait(browser, 20)
wait.until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))).click()

buscador = browser.find_element(By.NAME, "searchTerm")
buscador.send_keys("fruta")
sleep(2)
buscador = browser.find_element(By.CLASS_NAME, "search-button").click()
sleep(2)

data_eroski = []

browser.execute_script("window.scrollBy(0, window.innerHeight);")
sleep(10)

tiempo_limite_scroll = 12

# Obtener el tiempo actual
tiempo_inicial = time()

# Activar el scroll durante el tiempo límite
while (time() - tiempo_inicial) < tiempo_limite_scroll:
    browser.execute_script("window.scrollBy(0, window.innerHeight);")

#paseo y adicción de informacion a nuestra lista
soup = BeautifulSoup(browser.page_source, "html.parser")   
productos = soup.find_all("div", class_="col col-xs-12 col-sm-12 col-md-12 col-lg-12 product-item big-item")
for producto in productos:
    nombre = producto.find("h2", class_="product-title").text
    if 'tarrina' in nombre:
        nombre = 'None'
    else:   
        nombre = nombre.split()[0]
    try:
        precio = float(producto.find("span", class_="price-product").text.split()[0].replace(",", "."))
    except:
        precio = float(producto.find("span", class_="price-offer-now").text.split()[0].replace(",", "."))
        pass

    data_eroski.append({"nombre": nombre, "precio": precio, 'eroski':'eroski'})

browser.quit()

Airtable Eroski

In [213]:
API_KEY = "" # Usuario

BASE_ID = "app3rDVq7YZmHLPMn" # Base: Tabla API

TABLE_ID = "tbl6FCpAlIrAgkpWO" # Tabla: datos_1

airtable_base_url = "https://api.airtable.com/v0"

# Headers
headers = {"Authorization" : f"Bearer {API_KEY}",
           "Content-Type"  : "application/json"}

endpoint = f"{airtable_base_url}/{BASE_ID}/{TABLE_ID}"

records = []
for item in data_eroski:
    records_item = {'fields': {"Producto": item["nombre"], 'Precio': item['precio'],'Supermercado':item['eroski']}}
    records.append(records_item)
    
cantidad_records = len(records)
posicion = 0

while posicion < cantidad_records:
    records_utilizar = records[posicion : (posicion + 10)]
    records_dict = {"records" : records_utilizar}
    response = requests.post(url = endpoint, json = records_dict, headers = headers)

    if response.status_code != 200:
        print(response.text)
        break
    
    posicion += 10

## Supermercado Ahorramas

In [156]:
url_ahorramas = "https://www.ahorramas.com/"

chrome_driver = "chromedriver"

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
browser = webdriver.Chrome(options=options)
browser.get(url=url_ahorramas)

wait = WebDriverWait(browser, 20)

# Aceptar cookies y ruta de navegación.
try:
    wait.until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))).click()
except:
    pass

buscador = browser.find_element(By.NAME, "q")
buscador.send_keys("fruta")
buscador.send_keys(Keys.RETURN)

data_ahorramas = []

# Movimiento por la web 
num_clicks = 0
start_index = 22
items_por_pag = 12

while num_clicks < 6:
    try:
        more_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Más resultados') and not(@disabled)]")))
        more_button.click()
        num_clicks += 1
        start_index += items_por_pag
        sleep(2)

    except:
        pass

# Crear un objeto BeautifulSoup para analizar el HTML que ya se encuentra en la página y obtener los contenedores de los productos
soup = BeautifulSoup(browser.page_source, "html.parser")
contenedor_productos = soup.find_all("div", class_="tile-body")

#iteramos en los contenedores ya que se encuentran elementos con diferentes nombres
#Convertimos nombres a texto y precios a flotante
for element in contenedor_productos:
    try:
        precio = float(element.find("div", class_="unit-price-row price").text.split('(')[1].split('€')[0].replace(',','.'))
    except:
        precio = float(element.find("span", class_="value").text.split('€')[0].replace(',','.'))
        
    nombre = element.find("h2", class_="link product-name-gtm").text.split()[0]
    
# Agregar la información a la lista data_ahorramas    
    data_ahorramas.append({"nombre": nombre, "precio": precio, "ahorramas": "ahorramas"})


browser.quit()

Airtable Ahorramás

In [ ]:
API_KEY = "" # Usuario

BASE_ID = "app3rDVq7YZmHLPMn" # Base: Tabla API

TABLE_ID = "tbl6FCpAlIrAgkpWO" # Tabla: datos_1

airtable_base_url = "https://api.airtable.com/v0"

# Headers
headers = {"Authorization" : f"Bearer {API_KEY}",
           "Content-Type"  : "application/json"}

endpoint = f"{airtable_base_url}/{BASE_ID}/{TABLE_ID}"

records = []
for item in data_ahorramas:
    records_item = {'fields': {"Producto": item["nombre"], 'Precio': item['precio'],'Supermercado':item['ahorramas']}}
    records.append(records_item)
    
cantidad_records = len(records)
posicion = 0

while posicion < cantidad_records:
    records_utilizar = records[posicion : (posicion + 10)]
    records_dict = {"records" : records_utilizar}
    response = requests.post(url = endpoint, json = records_dict, headers = headers)

    if response.status_code != 200:
        print(response.text)
        break
    
    posicion += 10